In [2]:
import cv2
import mediapipe as mp

# Mediapipe 핸드 모듈 초기화
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 웹캠 캡처 초기화
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 프레임 색상 변환(BGR -> RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)
    
    # 손 인식 데이터가 있을 경우
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # 손 랜드마크를 화면에 그리기
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # 각 랜드마크 좌표 계산
            h, w, c = frame.shape  # 영상의 높이, 너비
            index_finger_tip = hand_landmarks.landmark[8]  # 검지 끝 (랜드마크 8번)
            cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            
            # 검지가 화면 위쪽 1/3에 위치할 때 네모 그리기
            if cy < h // 3:  # 화면 높이의 상단 1/3 기준
                cv2.rectangle(frame, (50, 50), (150, 150), (0, 255, 0), -1)
            
            # 검지 위치 표시
            cv2.putText(frame, f"Index Finger: {cx}, {cy}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # 영상 출력
    cv2.imshow("Hand Tracking", frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
